In [1]:
from electric_meter import *
from IPython.display import HTML

In [13]:
fname='uploads/was-excel.xlsx'
fname='uploads/mine-sep2023.csv'
#fname='uploads/1694442555_9_5acc46e3.csv'
#meter = read_data('data/meter-big.csv')
meter = read_data(fname)
#meter = pd.read_excel(fname, skiprows=11)

In [14]:
#meter = read_data('data/meter.csv')

costs, conf = compute_costs(meter)

In [15]:
costs

,no_discount,pazgas_daytime,pazgas_unlimited,pazgas_weekend,pazgas_nighttime,amisragas_unlimited,electra_power,electra_hitec,cellcom_flat,cellcom_home_office,cellcom_nighttime,taoz1,taoz2
timeperiod,,,,,,,,,,,,,
יולי 2023,492.794713,467.561011,468.154978,477.911818,475.768644,460.763057,468.154978,467.465945,468.154978,471.719673,474.227697,831.918762,658.848762
אוגוסט 2023,566.034147,534.704102,537.732439,549.896450,550.166944,529.241927,537.732439,537.252142,537.732439,538.298207,547.225861,934.830385,761.760385
ספטמבר 2023,186.562425,178.092242,177.234304,179.288672,181.639629,174.435868,177.234304,178.101348,177.234304,180.177807,181.769549,452.495363,279.425363


In [16]:
result = style_table(costs)
display(result)

,חחי,פזגז יום,פזגז ללא הגבלה,פזגז סופ״ש,פזגז לילה,אמישראגז,5% אלקטרה פאואר,אלקטרה הייטק,סלקום 5%,סלקום מהבית,סלקום לילה,תעו״ז חד-חודשי,תעו״ז דו-חודשי
תקופה,,,,,,,,,,,,,
יולי 2023,492.79,467.56,468.15,477.91,475.77,460.76,468.15,467.47,468.15,471.72,474.23,831.92,658.85
אוגוסט 2023,566.03,534.70,537.73,549.90,550.17,529.24,537.73,537.25,537.73,538.30,547.23,934.83,761.76
ספטמבר 2023,186.56,178.09,177.23,179.29,181.64,174.44,177.23,178.10,177.23,180.18,181.77,452.50,279.43


In [8]:
if False:
    cols = [costs.columns[i] for i in [0,1,5, 6]]
    display(style_table(costs[cols].copy()))

In [24]:
v = meter.query("timeperiod == '2023095'")

#meter.groupby('month').mean(numeric_only=True).plot()
#meter.groupby('hour').mean(numeric_only=True).plot()

In [25]:
if False:
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt

    # Create a pivot table for heatmap
    heatmap_data = meter.query("month == 9").pivot_table(index="wday", columns="hour", values="consumption", aggfunc="mean")

    # Create the heatmap using seaborn
    plt.figure(figsize=(10, 6))
    sns.heatmap(heatmap_data, annot=False, cmap="YlGnBu", fmt="")
    plt.title("Consumption Heatmap")
    plt.xlabel("Hour")
    plt.ylabel("Month")
    plt.show()
